<a href="https://colab.research.google.com/github/techbrothers3/Diabetes-prediction-kaggle/blob/main/Diabetes_Prediction_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [10]:
class ClassificationModel:
    def __init__(self, data_url, target_column):
        self.data_url = data_url
        self.target_column = target_column
        self.data = None
        self.models = {
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(),
            'Logistic Regression': LogisticRegression(),
            'Naive Bayes': GaussianNB()
        }
        self.scaler = StandardScaler()

    def load_data(self):
        self.data = pd.read_csv(self.data_url)

    def preprocess_data(self):
        X = self.data.drop([self.target_column], axis=1)
        y = self.data[self.target_column]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train = self.scaler.fit_transform(X_train)
        X_test = self.scaler.transform(X_test)

        return X_train, X_test, y_train, y_test

    def train_models(self, X_train, y_train):
        for model_name, model in self.models.items():
            model.fit(X_train, y_train)
            print(f"{model_name} trained successfully.")

    def evaluate_models(self, X_test, y_test):
        for model_name, model in self.models.items():
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            report = classification_report(y_test, y_pred)
            print(f"\n{model_name} Evaluation:")
            print(f"Accuracy: {accuracy:.4f}")
            print(f"Classification Report:\n{report}")

In [11]:
if __name__ == "__main__":

  diabetes_model = ClassificationModel("diabetes.csv", "Outcome")

  diabetes_model.load_data()

  X_train, X_test, y_train, y_test = diabetes_model.preprocess_data()

  diabetes_model.train_models(X_train, y_train)

  diabetes_model.evaluate_models(X_test, y_test)

KNN trained successfully.
SVM trained successfully.
Logistic Regression trained successfully.
Naive Bayes trained successfully.

KNN Evaluation:
Accuracy: 0.6948
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.80      0.77        99
           1       0.58      0.51      0.54        55

    accuracy                           0.69       154
   macro avg       0.66      0.65      0.66       154
weighted avg       0.69      0.69      0.69       154


SVM Evaluation:
Accuracy: 0.7338
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        99
           1       0.65      0.56      0.60        55

    accuracy                           0.73       154
   macro avg       0.71      0.70      0.70       154
weighted avg       0.73      0.73      0.73       154


Logistic Regression Evaluation:
Accuracy: 0.7532
Classification Report:
              precision    reca